In [7]:
!pip install -U langchain-community langchain-core langchain-groq
!pip install -U langchain-huggingface langchain-groq langchain-chroma langchain-community langchain-text-splitters

In [4]:
!pip install -U langchain-huggingface langchain-groq langchain-chroma langchain-community langchain

In [2]:
import requests
from bs4 import BeautifulSoup

def simple_scraper(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # سحب النصوص من الفقرات والعناوين
    text_data = [ p.get_text() for p in soup.find_all(['h1', 'h2', 'p']) ]
    full_text = "\n".join(text_data)

    with open("parks_info.txt", "w", encoding="utf-8") as f:
        f.write(full_text)
    print("✅ تم حفظ بيانات المتنزهات في ملف: parks_info.txt")

# مثال: رابط لمقال عن منتزهات (يمكنك تغييره لأي رابط آخر)
target_url = "https://it.wikipedia.org/wiki/Riserva_naturale_I_Giganti_della_Sila"
simple_scraper(target_url)

✅ تم حفظ بيانات المتنزهات في ملف: parks_info.txt


In [2]:
import os
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. إعداد المفتاح
os.environ["GROQ_API_KEY"] = "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"

# 2. تحميل البيانات (تأكد أن الملف موجود في Colab)
try:
    loader = TextLoader("parks_info.txt")
    docs = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    print("✅ تم تحميل البيانات بنجاح")
except Exception as e:
    print("❌ خطأ: تأكد من تشغيل خلية الـ Scraper أولاً لإنشاء الملف!")

# 3. قاعدة البيانات والـ Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

# 4. تصميم البرومبت (بالعربي)
template = """أنت مساعد ذكي متخصص في المتنزهات. استخدم المعلومات التالية فقط للإجابة:
{context}

السؤال: {question}
الإجابة:"""
prompt = ChatPromptTemplate.from_template(template)

# 5. بناء الـ Chain مع النموذج الجديد (Llama 3.3)
# استخدمنا llama-3.3-70b-versatile لأنه الأحدث والأقوى حالياً
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 6. التشغيل
question = "Qual è la definizione di parco nazionale in base al testo?"
response = rag_chain.invoke(question)

print("\n--- رد البوت ---\n")
print(response)

✅ تم تحميل البيانات بنجاح

--- رد البوت ---

Il testo non fornisce una definizione esplicita di "parco nazionale". Tuttavia, si può evincere che il Parco Nazionale della Sila è un'area protetta che comprende la Riserva naturale I Giganti della Sila e altre aree, e che è gestito dall'Ente Parco Nazionale della Sila. Inoltre, il testo menziona che la Riserva naturale I Giganti della Sila si trova all'interno del Parco Nazionale della Sila, il che suggerisce che il parco nazionale sia un'area più ampia che comprende diverse riserve naturali e aree protette.


In [3]:
import requests
from bs4 import BeautifulSoup

def sila_park_scraper():
    # روابط رسمية من موقع حديقة سيلا (الإيطالي أو الإنجليزي)
    urls = [
        "http://www.parcosila.it/it/visita-il-parco/i-centri-visita.html",
        "http://www.parcosila.it/it/natura/la-flora.html",
        "http://www.parcosila.it/it/natura/la-fauna.html"
    ]

    headers = {'User-Agent': 'Mozilla/5.0'}
    all_content = []

    for url in urls:
        print(f"جاري سحب البيانات من: {url}...")
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # التركيز على محتوى المقال الأساسي (عادة يكون داخل div معين)
            content = soup.find_all(['h1', 'h2', 'p'])
            for item in content:
                all_content.append(item.get_text().strip())

    with open("sila_data.txt", "w", encoding="utf-8") as f:
        f.write("\n\n".join(all_content))

    print(f"\n✅ تم تحديث قاعدة بيانات حديقة سيلا! (sila_data.txt)")

sila_park_scraper()

جاري سحب البيانات من: http://www.parcosila.it/it/visita-il-parco/i-centri-visita.html...
جاري سحب البيانات من: http://www.parcosila.it/it/natura/la-flora.html...
جاري سحب البيانات من: http://www.parcosila.it/it/natura/la-fauna.html...

✅ تم تحديث قاعدة بيانات حديقة سيلا! (sila_data.txt)


In [9]:
import os
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
# المسار الجديد والمضمون هنا:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# 1. إعداد المفتاح والبيئة
os.environ["GROQ_API_KEY"] = "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"
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# استبدل الجزء الخاص بتحميل البيانات (رقم 2) بهذا الكود المحسن:

try:
    with open("sila_data.txt", "r", encoding="utf-8") as f:
        content = f.read()

    if len(content) < 50:
        print("⚠️ تحذير: ملف sila_data.txt يبدو فارغاً أو صغيراً جداً!")

    # استخدام تقسيم أبسط لنضمن عدم ضياع النص
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

    # تحميل النص مباشرة
    from langchain_core.documents import Document
    docs = [Document(page_content=content)]
    splits = text_splitter.split_documents(docs)

    print(f"✅ تم تقسيم النص إلى {len(splits)} قطعة.")

    # بناء قاعدة البيانات
    vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # جلب أفضل 3 نتائج
    print("✅ تم بناء قاعدة بيانات حديقة سيلا بنجاح!")
except Exception as e:
    print(f"❌ خطأ في تحميل البيانات: {e}")

# 3. إعداد النموذج وتاريخ المحادثة
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.3)
chat_history = []

def ask_sila_expert(question):
    # جلب السياق
    relevant_docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    # بناء القالب
    prompt = ChatPromptTemplate.from_messages([
        ("system", "أنت مساعد سياحي خبير في حديقة سيلا الوطنية (Sila National Park). أجب باللغة العربية بناءً على السياق فقط:\n\n" + context),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}")
    ])

    # الربط والتشغيل
    chain = prompt | llm
    response = chain.invoke({"question": question, "history": chat_history})

    # تحديث الذاكرة
    chat_history.append(HumanMessage(content=question))
    chat_history.append(AIMessage(content=response.content))

    return response.content

# 4. بدء التجربة
print("\n--- البوت جاهز للأسئلة حول حديقة سيلا ---\n")
user_q = "Cosa posso vedere nella riserva I Giganti della Sila"
print(f"سؤالك: {user_q}")
print(f"البوت: {ask_sila_expert(user_q)}")

⚠️ تحذير: ملف sila_data.txt يبدو فارغاً أو صغيراً جداً!
✅ تم تقسيم النص إلى 0 قطعة.
❌ خطأ في تحميل البيانات: Expected Embeddings to be non-empty list or numpy array, got [] in upsert.

--- البوت جاهز للأسئلة حول حديقة سيلا ---

سؤالك: Cosa posso vedere nella riserva I Giganti della Sila
البوت: La riserva I Giganti della Sila è un'area naturale protetta situata nella provincia di Cosenza, in Italia. All'interno della riserva, potrai vedere:

* I 58 "Giganti della Sila" o "Giganti di Fallistro", pini larici ultracentenari di dimensioni maestose, che possono innalzarsi fino a 45 metri di altezza e avere un diametro alla base di circa due metri.
* Una pineta di Pino Laricio (Pinus nigra var. calabrica), impiantata nella prima metà del 1600.
* Altre specie di piante autoctone, come meli selvatici, faggi, castagni, pioppi tremuli e aceri montani.
* 7 esemplari centenari di aceri di montagna, che si collocano ai margini della riserva.
* Una varietà di fauna tipica delle zone silane e appennin

In [10]:
content_it = """
IL PARCO NAZIONALE DELLA SILA
Il Parco Nazionale della Sila, istituito nel 1997, si estende per 73.695 ettari in Calabria, toccando le province di Cosenza, Catanzaro e Crotone. È gestito dall'Ente Parco Nazionale della Sila con sede a Lorica.

RISERVA NATURALE I GIGANTI DELLA SILA (FALLISTRO)
La Riserva naturale guidata Biogenetica "I Giganti della Sila", situata in località Fallistro (comune di Spezzano della Sila), è un'area protetta che ospita 58 esemplari di pini larici (Pinus nigra laricio) ultracentenari. Questi alberi hanno un'età stimata di circa 350-400 anni.

CARATTERISTICHE DEI GIGANTI
- Altezza: Gli esemplari più imponenti raggiungono i 45 metri di altezza.
- Diametro: Molti tronchi superano i 2 metri di diametro alla base.
- Specie: Prevalenza di Pino Laricio, ma sono presenti anche esemplari di acero montano.

INFORMAZIONI PER IL VISITATORE (FAI)
La gestione operativa della Riserva è affidata al FAI (Fondo Ambiente Italiano) dal 2016.
- Orari di apertura: Da maggio a ottobre la riserva è aperta tutti i giorni (ore 10:00 - 18:00). Da novembre ad aprile l'apertura è limitata ai fine settimana o su prenotazione per gruppi.
- Accessibilità: È presente un percorso didattico accessibile a piedi.

FLORA E FAUNA DEL PARCO
La flora comprende il pino laricio, il faggio e l'abete bianco. La fauna è caratterizzata dalla presenza del lupo appenninico (simbolo del parco), del capriolo, del cervo, dello scoiattolo nero calabrese e del gatto selvatico. Tra i rapaci si trovano la poiana e il nibbio reale.
"""

with open("sila_data.txt", "w", encoding="utf-8") as f:
    f.write(content_it)

print("✅ Il file 'sila_data.txt' è stato creato con informazioni ufficiali in italiano.")

✅ Il file 'sila_data.txt' è stato creato con informazioni ufficiali in italiano.


In [11]:
import os
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# 1. Setting up the environment
# تأكد من وضع مفتاح Groq الخاص بك هنا
os.environ["GROQ_API_KEY"] = "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"
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Loading the Official Italian Data
with open("sila_data.txt", "r", encoding="utf-8") as f:
    ita_text = f.read()

# Splitting the Italian text into manageable chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=60)
docs = [Document(page_content=ita_text)]
splits = text_splitter.split_documents(docs)

# 3. Creating the Vector Store (Database)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 4. The RAG Expert Function (English Response)
def ask_sila_expert_en(question):
    # Retrieve relevant Italian context
    context_docs = retriever.invoke(question)
    context = "\n\n".join([d.page_content for d in context_docs])

    # Instructions to the LLM: Read Italian Context -> Respond in English
    prompt = f"""
    You are an official tourist guide for the Sila National Park.
    Using the provided Italian context below, answer the user's question.
    You must respond in ENGLISH.

    Italian Context:
    {context}

    User Question: {question}
    Expert Response (English):"""

    llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.2)
    return llm.invoke(prompt).content

# 5. Testing the bot in English
print("\n--- Sila Park Expert Bot (English Response) ---")
test_q = "What are the main characteristics and height of the Giants of Sila?"
print(f"Question: {test_q}")
print(f"Response:\n{ask_sila_expert_en(test_q)}")


--- Sila Park Expert Bot (English Response) ---
Question: What are the main characteristics and height of the Giants of Sila?
Response:
The Giants of Sila, also known as the Giants of Fallistro, are a group of 58 ultracentenarian pine trees, specifically Pinus nigra laricio, that can be found in the Sila National Park in Calabria, Italy. The main characteristics of these trees are their impressive size and age, with some of them estimated to be around 350-400 years old.

In terms of height, the Giants of Sila can reach up to 45 meters (147.6 feet) tall, with a diameter of approximately 2 meters (6.6 feet) at the base of the trunk. While they are not as tall as the sequoias found in North America, which can reach heights of over 80 meters (262.5 feet), the Giants of Sila are still an impressive sight to behold, forming a natural colonnade that is unique in the region.

It's worth noting that the Giants of Sila are not only remarkable for their size and age but also for their ability to

In [12]:
try:
    import gradio as gr
except:
    !pip install gradio
    import gradio as gr

def chat_interface(message, history):
    return ask_sila_expert_en(message)

# Launching a clean UI
demo = gr.ChatInterface(
    fn=chat_interface,
    title="Sila National Park Expert 🌲",
    description="Official Virtual Assistant for Parco Nazionale della Sila. Ask me about trails, the Giants, and opening hours.",
    examples=["Who manages the Giant Trees?", "What is the best time to visit?", "Tell me about the Sila Wolf."],
    theme="soft"
)

demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bd18a7d1756aebe43c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
